In [22]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
from collections import Counter
import pandas as pd
import random
import matplotlib.pyplot as plt
import itertools

In [23]:
def load_data(filename, genrename):
    file = pd.read_csv(filename, sep='\t')
    file = file[file['genre'] == genrename]
    data = [[file.iloc[i,0],file.iloc[i,1]] for i in range(len(file.index))]
    file['label'] = file['label'].replace(['entailment', 'contradiction', 'neutral'], [0,1,2])
    labels = file['label']
#     file['genre'] = file['genre'].replace(['telephone', 'fiction', 'slate', 'government', 'travel'], [0,1,2,3,4])
#     genres = file['genre']
    return data, labels

In [24]:
val_data_1, val_targets_1 = load_data('mnli_val.tsv', 'telephone')
val_data_2, val_targets_2 = load_data('mnli_val.tsv', 'fiction')
val_data_3, val_targets_3 = load_data('mnli_val.tsv', 'slate')
val_data_4, val_targets_4 = load_data('mnli_val.tsv', 'government')
val_data_5, val_targets_5 = load_data('mnli_val.tsv', 'travel')

# file = pd.read_csv('mnli_train.tsv', sep='\t')
# file = file[:10000]
# file['label']= file['label'].replace(['entailment', 'contradiction', 'neutral'], [0,1,2])
# file.iloc[5,0]

In [25]:
val_data_tokens_1 = [[val_data_1[i][j].split() for j in range(2)] for i in range(len(val_data_1))]
val_data_tokens_2 = [[val_data_2[i][j].split() for j in range(2)] for i in range(len(val_data_2))]
val_data_tokens_3 = [[val_data_3[i][j].split() for j in range(2)] for i in range(len(val_data_3))]
val_data_tokens_4 = [[val_data_4[i][j].split() for j in range(2)] for i in range(len(val_data_4))]
val_data_tokens_5 = [[val_data_5[i][j].split() for j in range(2)] for i in range(len(val_data_5))]


with open('wiki-news-300d-1M.vec') as f:
    lines = []
    for i in range(50001):
        line = f.readline()
        v = line.split()
        for j in range(1,len(v)):
            v[j] = float(v[j])
        lines.append(v)
        
lines.remove(lines[0])  

word_dict = {}
word_dict['PAD'] = 0
word_dict['UNK'] = 1

embed = [[0 for i in range(300)],[0 for i in range(300)]]

for i in range(len(lines)):
    word_dict['PAD'] = 0
    word_dict['UNK'] = 1
    word_dict[lines[i][0]] = i+2
    embed.append(lines[i][1:])
    
embedding_matrix = np.matrix(embed)

id2token = []
for word in word_dict.keys():
    id2token.append(word)
token2id = word_dict

UNK_IDX = 1
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        sublist = []
        for i in range(2):
            index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens[i]]
            sublist.append(index_list)
        indices_data.append(sublist)
    return indices_data

val_data_indices_1 = token2index_dataset(val_data_tokens_1)
val_data_indices_2 = token2index_dataset(val_data_tokens_2)
val_data_indices_3 = token2index_dataset(val_data_tokens_3)
val_data_indices_4 = token2index_dataset(val_data_tokens_4)
val_data_indices_5 = token2index_dataset(val_data_tokens_5)


In [26]:
MAX_SENTENCE_LENGTH = 50
class MNLIDataset(Dataset):
    def __init__(self, data_list, target_list):
        self.data_list = data_list
        self.target_list = target_list
        self.target_list.index = range(len(self.target_list))
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key):

        """
        Triggered when you call dataset[i]
        """
        
        token_idx_1 = self.data_list[key][0][:MAX_SENTENCE_LENGTH]
        token_idx_2 = self.data_list[key][1][:MAX_SENTENCE_LENGTH]
        token_idx = [token_idx_1, token_idx_2]
        label = self.target_list[key]
        return [token_idx, [len(token_idx_1), len(token_idx_2)], label]

def MNLI_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []

    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
    # padding
    for datum in batch:
        padded_vec_1 = np.pad(np.array(datum[0][0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1][0])), 
                                mode="constant", constant_values=0)
        padded_vec_2 = np.pad(np.array(datum[0][1]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1][1])), 
                                mode="constant", constant_values=0)
#         sublist = ' '.join(padded_vec_2)
#         data_list.append(padded_vec_1)
#         data_list.append(padded_vec_2)
        data_list.append(list(padded_vec_1) + list(padded_vec_2))
    return [torch.from_numpy(np.array(data_list)), torch.LongTensor(length_list), torch.LongTensor(label_list)]

In [27]:
BATCH_SIZE = 40

val_dataset_1 = MNLIDataset(val_data_indices_1, val_targets_1)
val_dataset_2 = MNLIDataset(val_data_indices_2, val_targets_2)
val_dataset_3 = MNLIDataset(val_data_indices_3, val_targets_3)
val_dataset_4 = MNLIDataset(val_data_indices_4, val_targets_4)
val_dataset_5 = MNLIDataset(val_data_indices_5, val_targets_5)

val_loader_1 = torch.utils.data.DataLoader(dataset=val_dataset_1, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=MNLI_collate_func,
                                           shuffle=True)
val_loader_2 = torch.utils.data.DataLoader(dataset=val_dataset_2, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=MNLI_collate_func,
                                           shuffle=True)
val_loader_3 = torch.utils.data.DataLoader(dataset=val_dataset_3, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=MNLI_collate_func,
                                           shuffle=True)
val_loader_4 = torch.utils.data.DataLoader(dataset=val_dataset_4, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=MNLI_collate_func,
                                           shuffle=True)
val_loader_5 = torch.utils.data.DataLoader(dataset=val_dataset_5, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=MNLI_collate_func,
                                           shuffle=True)

In [28]:
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data, lengths, labels in loader:
        data_batch, lengths_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, lengths_batch), dim=1)
#         print(model(data_batch, lengths_batch))
#         print(outputs)
        predicted = outputs.max(1, keepdim=True)[1]

        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

In [29]:
weight = torch.FloatTensor(embed)

class Temp(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes, vocab_size):
        # RNN Accepts the following hyperparams:
        # emb_size: Embedding Size
        # hidden_size: Hidden Size of layer in RNN
        # num_layers: number of layers in RNN
        # num_classes: number of output classes
        # vocab_size: vocabulary size
        super(Temp, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding.from_pretrained(weight)
#         self.rnn = nn.RNN(emb_size, hidden_size, num_layers, batch_first=True)#creating RNN in pytorch
        self.bi_gru1 = nn.GRU(emb_size, hidden_size, num_layers=1, batch_first=True,bidirectional=True)
        self.bi_gru2 = nn.GRU(emb_size, hidden_size, num_layers=1, batch_first=True,bidirectional=True)

        # 2 FC layers
        self.linear1 = nn.Linear(4 * hidden_size, 100)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(100, 3)


    def init_hidden(self, batch_size):
        # Function initializes the activation of recurrent neural net at timestep 0
        # Needs to be in format (num_layers, batch_size, hidden_size)
        hidden = torch.zeros(self.num_layers*2, batch_size, self.hidden_size)
        return hidden

    def forward(self, x, lengths):
        # reset hidden state

        batch_size, seq_len = x.size()   
        #main part of rnn
        self.hidden1 = self.init_hidden(batch_size) # old size = batch_size 
        self.hidden2 = self.init_hidden(batch_size) # old size = batch_size 

        #######################################################
        # code here
        
        # get embedding of characters
        embed = self.embedding(x)
        embed.detach()
        # sort the sequence
        sorted_seq_lengths1, indices1 = torch.sort(lengths[:, 0], descending=True)
        sorted_seq_lengths2, indices2 = torch.sort(lengths[:, 1], descending=True)
        
        embed_1 = embed[:, :MAX_SENTENCE_LENGTH, :][indices1]
        embed_2 = embed[:, MAX_SENTENCE_LENGTH:, :][indices2]
        
        
        embed_1 = torch.nn.utils.rnn.pack_padded_sequence(embed_1, 
                                                          sorted_seq_lengths1.numpy(), 
                                                          batch_first=True)
        
        embed_2 = torch.nn.utils.rnn.pack_padded_sequence(embed_2, 
                                                          sorted_seq_lengths2.numpy(), 
                                                          batch_first=True)

        _, desorted_indices1 = torch.sort(indices1, descending=False)
        _, desorted_indices2 = torch.sort(indices2, descending=False)
        
        #bi-directional GRU
        bi_output1, self.hidden1 = self.bi_gru1(embed_1,self.hidden1)
        bi_output2, self.hidden2 = self.bi_gru2(embed_2,self.hidden2)
        # rearrange output into correct order
#         print(self.hidden1.size())
        self.hidden1 = self.hidden1[:, desorted_indices1, :]
        self.hidden2 = self.hidden2[:, desorted_indices2, :]

        
        bi_gru_out = torch.cat((torch.cat([self.hidden1[0], self.hidden1[1]], dim=-1), 
                                torch.cat([self.hidden2[0], self.hidden2[1]], dim=-1)), dim=1)

        # FC layers
        rnn_out = self.linear1(bi_gru_out)
        activate = self.activation(rnn_out)
        logits = self.linear2(activate)
        
        return logits


In [39]:
weight = torch.FloatTensor(embed)

class CNN(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes, vocab_size, ks):

        super(CNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
#         self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        self.embedding = nn.Embedding.from_pretrained(weight)
    
        self.conv1 = nn.Conv1d(emb_size, hidden_size, kernel_size=ks, padding=int((ks-1)/2))
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=ks, padding=int((ks-1)/2))
        
        self.conv3 = nn.Conv1d(emb_size, hidden_size, kernel_size=ks, padding=int((ks-1)/2))
        self.conv4 = nn.Conv1d(hidden_size, hidden_size, kernel_size=ks, padding=int((ks-1)/2))
        
        self.maxpool = nn.MaxPool1d(kernel_size=MAX_SENTENCE_LENGTH)
        
        self.linear1 = nn.Linear(hidden_size*2, 128)
        self.dropout = nn.Dropout(p=0.25)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(128, num_classes)

    def forward(self, x, lengths):
        batch_size, seq_len = x.size()

        embed = self.embedding(x)
        embed = self.dropout(embed)
        embed_1 = embed[:, :MAX_SENTENCE_LENGTH, :] #100,50,300
        embed_2 = embed[:, MAX_SENTENCE_LENGTH:, :]

        hidden_1 = self.conv1(embed_1.transpose(1,2)).transpose(1, 2)
        hidden_1 = F.relu(hidden_1.contiguous().view(-1, hidden_1.size(-1))).view(batch_size, MAX_SENTENCE_LENGTH, hidden_1.size(-1))
        
#         print(hidden_1.size())
        hidden_1 = self.conv2(hidden_1.transpose(1,2)).transpose(1,2)
        hidden_1 = F.relu(hidden_1.contiguous().view(-1, hidden_1.size(-1))).view(batch_size, MAX_SENTENCE_LENGTH, hidden_1.size(-1))
        
        hidden_2 = self.conv3(embed_2.transpose(1,2)).transpose(1, 2)
        hidden_2 = F.relu(hidden_2.contiguous().view(-1, hidden_2.size(-1))).view(batch_size, MAX_SENTENCE_LENGTH, hidden_2.size(-1))

        hidden_2 = self.conv4(hidden_2.transpose(1,2)).transpose(1,2)
        hidden_2 = F.relu(hidden_2.contiguous().view(-1, hidden_2.size(-1))).view(batch_size, MAX_SENTENCE_LENGTH, hidden_2.size(-1))

        # max pool
        hidden_1 = self.maxpool(hidden_1.transpose(1, 2)).transpose(1, 2)
        hidden_1 = torch.squeeze(hidden_1, dim=1)
        hidden_2 = self.maxpool(hidden_2.transpose(1, 2)).transpose(1, 2)
        hidden_2 = torch.squeeze(hidden_2, dim=1)
        
        hidden = torch.cat((hidden_1, hidden_2), dim=1)
#         print(hidden.size())
        
        hidden = self.linear1(hidden)
        hidden = self.dropout(hidden)
        hidden = self.activation(hidden)
        logits = self.linear2(hidden)
        return logits

In [131]:
model = Temp(emb_size=300, hidden_size=600, num_layers=1, num_classes=3, vocab_size=len(id2token))
model.load_state_dict(torch.load('rnnmodel.pt', map_location='cpu'))
# model.eval()

print("Test Acc for “telephone” = {} ".format(test_model(val_loader_1, model)))
print("Test Acc for “fiction” = {} ".format(test_model(val_loader_2, model)))
print("Test Acc for “slate” = {} ".format(test_model(val_loader_3, model)))
print("Test Acc for “government” = {} ".format(test_model(val_loader_4, model)))
print("Test Acc for “travel” = {} ".format(test_model(val_loader_5, model)))


Test Acc for “telephone” = 46.865671641791046 
Test Acc for “fiction” = 46.4321608040201 
Test Acc for “slate” = 42.61477045908184 
Test Acc for “government” = 47.93307086614173 
Test Acc for “travel” = 45.21384928716904 


In [40]:
model2 = CNN(emb_size=300, hidden_size=500, num_layers=2, num_classes=3, vocab_size=len(id2token), ks=3)
model2.load_state_dict(torch.load('cnnmodel.pt', map_location='cpu'))

In [41]:
print("Test Acc for “telephone” = {} ".format(test_model(val_loader_1, model2)))
print("Test Acc for “fiction” = {} ".format(test_model(val_loader_2, model2)))
print("Test Acc for “slate” = {} ".format(test_model(val_loader_3, model2)))
print("Test Acc for “government” = {} ".format(test_model(val_loader_4, model2)))
print("Test Acc for “travel” = {} ".format(test_model(val_loader_5, model2)))

Test Acc for “telephone” = 43.681592039801 
Test Acc for “fiction” = 42.814070351758794 
Test Acc for “slate” = 43.11377245508982 
Test Acc for “government” = 42.42125984251968 
Test Acc for “travel” = 44.19551934826884 


In [48]:
len(train_data_1)

4270

In [43]:
train_data_1, train_targets_1 = load_data('mnli_train.tsv', 'telephone')
train_data_2, train_targets_2 = load_data('mnli_train.tsv', 'fiction')
train_data_3, train_targets_3 = load_data('mnli_train.tsv', 'slate')
train_data_4, train_targets_4 = load_data('mnli_train.tsv', 'government')
train_data_5, train_targets_5 = load_data('mnli_train.tsv', 'travel')

train_data_tokens_1 = [[train_data_1[i][j].split() for j in range(2)] for i in range(len(train_data_1))]
train_data_tokens_2 = [[train_data_2[i][j].split() for j in range(2)] for i in range(len(train_data_2))]
train_data_tokens_3 = [[train_data_3[i][j].split() for j in range(2)] for i in range(len(train_data_3))]
train_data_tokens_4 = [[train_data_4[i][j].split() for j in range(2)] for i in range(len(train_data_4))]
train_data_tokens_5 = [[train_data_5[i][j].split() for j in range(2)] for i in range(len(train_data_5))]

train_data_indices_1 = token2index_dataset(train_data_tokens_1)
train_data_indices_2 = token2index_dataset(train_data_tokens_2)
train_data_indices_3 = token2index_dataset(train_data_tokens_3)
train_data_indices_4 = token2index_dataset(train_data_tokens_4)
train_data_indices_5 = token2index_dataset(train_data_tokens_5)

train_dataset_1 = MNLIDataset(train_data_indices_1, train_targets_1)
train_dataset_2 = MNLIDataset(train_data_indices_2, train_targets_2)
train_dataset_3 = MNLIDataset(train_data_indices_3, train_targets_3)
train_dataset_4 = MNLIDataset(train_data_indices_4, train_targets_4)
train_dataset_5 = MNLIDataset(train_data_indices_5, train_targets_5)

train_loader_1 = torch.utils.data.DataLoader(dataset=train_dataset_1, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=MNLI_collate_func,
                                           shuffle=True)
train_loader_2 = torch.utils.data.DataLoader(dataset=train_dataset_2, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=MNLI_collate_func,
                                           shuffle=True)
train_loader_3 = torch.utils.data.DataLoader(dataset=train_dataset_3, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=MNLI_collate_func,
                                           shuffle=True)
train_loader_4 = torch.utils.data.DataLoader(dataset=train_dataset_4, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=MNLI_collate_func,
                                           shuffle=True)
train_loader_5 = torch.utils.data.DataLoader(dataset=train_dataset_5, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=MNLI_collate_func,
                                           shuffle=True)

In [83]:
# model = Temp(emb_size=300, hidden_size=600, num_layers=1, num_classes=3, vocab_size=len(id2token))
# model.load_state_dict(torch.load('rnnmodel.pt', map_location='cpu'))
num_epochs = 3
learning_rate = 0.0003

criterion = torch.nn.CrossEntropyLoss()
parameters = itertools.filterfalse(lambda p: p.requires_grad == False, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# Train the model
total_step = len(train_loader_1)


loss_l = []
acc_l = []
for epoch in range(num_epochs):
    scheduler.step()
    for i, (data, lengths, labels) in enumerate(train_loader_1):
        model.train()
        optimizer.zero_grad()
        # Forward pass
        outputs = model(data,lengths)
        loss = criterion(outputs, labels)
        if i % 100 == 0:
            loss_l.append(loss.item())

        # Backward and optimize
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
#         if i > 0 and i % 100 == 0:
#             # validate
#             val_acc = test_model(val_loader_1, model)
#             print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
#                        epoch+1, num_epochs, i+1, len(train_loader_1), val_acc))
#             acc_l.append(val_acc)

print('Tuning on telephone model')
val_acc_1 = test_model(val_loader_1, model)
val_acc_2 = test_model(val_loader_2, model)
val_acc_3 = test_model(val_loader_3, model)
val_acc_4 = test_model(val_loader_4, model)
val_acc_5 = test_model(val_loader_5, model)
print('For genre : telephone, accuracy = ' + str(val_acc_1))
print('For genre : fiction, accuracy = ' + str(val_acc_2))
print('For genre : slate, accuracy = ' + str(val_acc_3))
print('For genre : government, accuracy = ' + str(val_acc_4))
print('For genre : travel, accuracy = ' + str(val_acc_5))

Tuning on telephone model
For genre : telephone, accuracy = 51.74129353233831
For genre : fiction, accuracy = 50.050251256281406
For genre : slate, accuracy = 47.10578842315369
For genre : government, accuracy = 46.94881889763779
For genre : travel, accuracy = 49.89816700610998


In [68]:
model = Temp(emb_size=300, hidden_size=600, num_layers=1, num_classes=3, vocab_size=len(id2token))
model.load_state_dict(torch.load('rnnmodel.pt', map_location='cpu'))
num_epochs = 3
learning_rate = 0.0003

criterion = torch.nn.CrossEntropyLoss()
parameters = itertools.filterfalse(lambda p: p.requires_grad == False, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# Train the model
total_step = len(train_loader_2)


loss_l = []
acc_l = []
for epoch in range(num_epochs):
    scheduler.step()
    for i, (data, lengths, labels) in enumerate(train_loader_2):
        model.train()
        optimizer.zero_grad()
        # Forward pass
        outputs = model(data,lengths)
        loss = criterion(outputs, labels)
        if i % 100 == 0:
            loss_l.append(loss.item())

        # Backward and optimize
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
#         if i > 0 and i % 100 == 0:
#             # validate
#             val_acc = test_model(val_loader_1, model)
#             print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
#                        epoch+1, num_epochs, i+1, len(train_loader_1), val_acc))
#             acc_l.append(val_acc)

print('Tuning on fiction model')
val_acc_1 = test_model(val_loader_1, model)
val_acc_2 = test_model(val_loader_2, model)
val_acc_3 = test_model(val_loader_3, model)
val_acc_4 = test_model(val_loader_4, model)
val_acc_5 = test_model(val_loader_5, model)
print('For genre : telephone, accuracy = ' + str(val_acc_1))
print('For genre : fiction, accuracy = ' + str(val_acc_2))
print('For genre : slate, accuracy = ' + str(val_acc_3))
print('For genre : government, accuracy = ' + str(val_acc_4))
print('For genre : travel, accuracy = ' + str(val_acc_5))

Tuning on fiction model
For genre : telephone, accuracy = 49.75124378109453
For genre : fiction, accuracy = 50.25125628140704
For genre : slate, accuracy = 46.20758483033932
For genre : government, accuracy = 47.539370078740156
For genre : travel, accuracy = 47.14867617107943


In [132]:
# model = Temp(emb_size=300, hidden_size=600, num_layers=1, num_classes=3, vocab_size=len(id2token))
# model.load_state_dict(torch.load('rnnmodel.pt', map_location='cpu'))
num_epochs = 3
learning_rate = 0.0003

criterion = torch.nn.CrossEntropyLoss()
parameters = itertools.filterfalse(lambda p: p.requires_grad == False, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# Train the model
total_step = len(train_loader_3)


loss_l = []
acc_l = []
for epoch in range(num_epochs):
    scheduler.step()
    for i, (data, lengths, labels) in enumerate(train_loader_3):
        model.train()
        optimizer.zero_grad()
        # Forward pass
        outputs = model(data,lengths)
        loss = criterion(outputs, labels)
        if i % 100 == 0:
            loss_l.append(loss.item())

        # Backward and optimize
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
#         if i > 0 and i % 100 == 0:
#             # validate
#             val_acc = test_model(val_loader_1, model)
#             print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
#                        epoch+1, num_epochs, i+1, len(train_loader_1), val_acc))
#             acc_l.append(val_acc)

print('Tuning on slate model')
val_acc_1 = test_model(val_loader_1, model)
val_acc_2 = test_model(val_loader_2, model)
val_acc_3 = test_model(val_loader_3, model)
val_acc_4 = test_model(val_loader_4, model)
val_acc_5 = test_model(val_loader_5, model)
print('For genre : telephone, accuracy = ' + str(val_acc_1))
print('For genre : fiction, accuracy = ' + str(val_acc_2))
print('For genre : slate, accuracy = ' + str(val_acc_3))
print('For genre : government, accuracy = ' + str(val_acc_4))
print('For genre : travel, accuracy = ' + str(val_acc_5))

Tuning on slate model
For genre : telephone, accuracy = 48.159203980099505
For genre : fiction, accuracy = 49.14572864321608
For genre : slate, accuracy = 47.405189620758485
For genre : government, accuracy = 50.88582677165354
For genre : travel, accuracy = 47.55600814663951


In [70]:
model = Temp(emb_size=300, hidden_size=600, num_layers=1, num_classes=3, vocab_size=len(id2token))
model.load_state_dict(torch.load('rnnmodel.pt', map_location='cpu'))
num_epochs = 3
learning_rate = 0.0003

criterion = torch.nn.CrossEntropyLoss()
parameters = itertools.filterfalse(lambda p: p.requires_grad == False, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# Train the model
total_step = len(train_loader_4)


loss_l = []
acc_l = []
for epoch in range(num_epochs):
    scheduler.step()
    for i, (data, lengths, labels) in enumerate(train_loader_4):
        model.train()
        optimizer.zero_grad()
        # Forward pass
        outputs = model(data,lengths)
        loss = criterion(outputs, labels)
        if i % 100 == 0:
            loss_l.append(loss.item())

        # Backward and optimize
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
#         if i > 0 and i % 100 == 0:
#             # validate
#             val_acc = test_model(val_loader_1, model)
#             print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
#                        epoch+1, num_epochs, i+1, len(train_loader_1), val_acc))
#             acc_l.append(val_acc)

print('Tuning on government model')
val_acc_1 = test_model(val_loader_1, model)
val_acc_2 = test_model(val_loader_2, model)
val_acc_3 = test_model(val_loader_3, model)
val_acc_4 = test_model(val_loader_4, model)
val_acc_5 = test_model(val_loader_5, model)
print('For genre : telephone, accuracy = ' + str(val_acc_1))
print('For genre : fiction, accuracy = ' + str(val_acc_2))
print('For genre : slate, accuracy = ' + str(val_acc_3))
print('For genre : government, accuracy = ' + str(val_acc_4))
print('For genre : travel, accuracy = ' + str(val_acc_5))

Tuning on government model
For genre : telephone, accuracy = 46.666666666666664
For genre : fiction, accuracy = 48.54271356783919
For genre : slate, accuracy = 47.405189620758485
For genre : government, accuracy = 53.44488188976378
For genre : travel, accuracy = 48.37067209775967


In [79]:
model = Temp(emb_size=300, hidden_size=600, num_layers=1, num_classes=3, vocab_size=len(id2token))
model.load_state_dict(torch.load('rnnmodel.pt', map_location='cpu'))
num_epochs = 3
learning_rate = 0.0003

criterion = torch.nn.CrossEntropyLoss()
parameters = itertools.filterfalse(lambda p: p.requires_grad == False, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# Train the model
total_step = len(train_loader_5)


loss_l = []
acc_l = []
for epoch in range(num_epochs):
    scheduler.step()
    for i, (data, lengths, labels) in enumerate(train_loader_5):
        model.train()
        optimizer.zero_grad()
        # Forward pass
        outputs = model(data,lengths)
        loss = criterion(outputs, labels)
        if i % 100 == 0:
            loss_l.append(loss.item())

        # Backward and optimize
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
#         if i > 0 and i % 100 == 0:
#             # validate
#             val_acc = test_model(val_loader_1, model)
#             print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
#                        epoch+1, num_epochs, i+1, len(train_loader_1), val_acc))
#             acc_l.append(val_acc)

print('Tuning on travel model')
val_acc_1 = test_model(val_loader_1, model)
val_acc_2 = test_model(val_loader_2, model)
val_acc_3 = test_model(val_loader_3, model)
val_acc_4 = test_model(val_loader_4, model)
val_acc_5 = test_model(val_loader_5, model)
print('For genre : telephone, accuracy = ' + str(val_acc_1))
print('For genre : fiction, accuracy = ' + str(val_acc_2))
print('For genre : slate, accuracy = ' + str(val_acc_3))
print('For genre : government, accuracy = ' + str(val_acc_4))
print('For genre : travel, accuracy = ' + str(val_acc_5))

Tuning on travel model
For genre : telephone, accuracy = 49.55223880597015
For genre : fiction, accuracy = 48.14070351758794
For genre : slate, accuracy = 45.10978043912176
For genre : government, accuracy = 50.88582677165354
For genre : travel, accuracy = 53.462321792260695
